In [1]:
import pandas as pd

In [6]:
data = pd.read_csv("../data/raw/drugsComTest_raw.tsv", sep="\t").drop("Unnamed: 0", axis=1)

In [11]:
data.head()

,drugName,condition,review,rating,date,usefulCount
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [26]:
data = data[data['condition'].str.contains("users found this comment helpful") == False]

In [34]:
data.groupby(["condition"]).count().sort_values(['drugName'], ascending=False)

,drugName,review,rating,date,usefulCount
condition,,,,,
Birth Control,9648,9648,9648,9648,9648
Depression,3095,3095,3095,3095,3095
Pain,2100,2100,2100,2100,2100
Anxiety,1908,1908,1908,1908,1908
Acne,1847,1847,1847,1847,1847
...,...,...,...,...,...
Dermatophytosis,1,1,1,1,1
Stomach Cance,1,1,1,1,1
Nonoccupational Exposure,1,1,1,1,1
